In [44]:
import pandas as pd
import numpy as np

# Load the data
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
imdb_movies = pd.read_csv('imdb_movies.csv')

In [38]:
imdb_movies.isna().sum()

names          0
date_x         0
score          0
genre         85
overview       0
crew          56
orig_title     0
status         0
orig_lang      0
budget_x       0
revenue        0
country        0
dtype: int64

In [39]:
movies.drop_duplicates()
ratings.drop_duplicates()
movies.describe()

,movieId
count,9742.000000
mean,42200.353623
std,52160.494854
min,1.000000
25%,3248.250000
50%,7300.000000
75%,76232.000000
max,193609.000000


In [40]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [41]:
import re

def remove_text_in_brackets(input_string):
    # Use a regular expression to match text within brackets and remove it
    result = re.sub(r'\([^)]*\)', '', input_string)
    result = result.strip() 
    return result

In [46]:
movies.loc[movies['title'] == 'Black Mirror']

,movieId,title,genres
9611,176601,Black Mirror,(no genres listed)


In [48]:
movies['genres'] = movies['genres'].str.split('|')
# movies['genres'] = movies['genres'].fillna("").astype('str')
movies['year'] = movies['title'].str.extract('.*\((.*)\).*', expand=True)
movies['title'] = movies['title'].apply(lambda x: remove_text_in_brackets(x))

In [49]:
movies.dropna(subset=['year'], inplace=True)

In [50]:
# change name of column
imdb_movies.rename(columns={'names':'title'}, inplace=True)
imdb_movies.drop(['orig_title'], axis=1, inplace=True)
imdb_movies.head()

,title,date_x,score,genre,overview,crew,status,orig_lang,budget_x,revenue,country
0,Creed III,03/02/2023,73.0,"Drama, Action","After dominating the boxing world, Adonis Cree...","Michael B. Jordan, Adonis Creed, Tessa Thompso...",Released,English,75000000.0,2.716167e+08,AU
1,Avatar: The Way of Water,12/15/2022,78.0,"Science Fiction, Adventure, Action",Set more than a decade after the events of the...,"Sam Worthington, Jake Sully, Zoe Saldaña, Neyt...",Released,English,460000000.0,2.316795e+09,AU
2,The Super Mario Bros. Movie,04/05/2023,76.0,"Animation, Adventure, Family, Fantasy, Comedy","While working underground to fix a water main,...","Chris Pratt, Mario (voice), Anya Taylor-Joy, P...",Released,English,100000000.0,7.244590e+08,AU
3,Mummies,01/05/2023,70.0,"Animation, Comedy, Family, Adventure, Fantasy","Through a series of unfortunate events, three ...","Óscar Barberán, Thut (voice), Ana Esther Albor...",Released,"Spanish, Castilian",12300000.0,3.420000e+07,AU
4,Supercell,03/17/2023,61.0,Action,Good-hearted teenager William always lived in ...,"Skeet Ulrich, Roy Cameron, Anne Heche, Dr Quin...",Released,English,77000000.0,3.409420e+08,US


In [51]:
movies['year'] = pd.to_numeric(movies['year'], errors='coerce')

In [52]:
movies['year'].describe()

count    9729.000000
mean     1994.613629
std        18.535219
min      1902.000000
25%      1988.000000
50%      1999.000000
75%      2008.000000
max      2018.000000
Name: year, dtype: float64

In [53]:
movies.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995.0
4,5,Father of the Bride Part II,[Comedy],1995.0


In [54]:
user_freq = ratings[['userId', 'movieId']].groupby('userId').count().reset_index()
user_freq.columns = ['userId', 'n_ratings']
user_freq

,userId,n_ratings
0,1,232
1,2,29
2,3,39
3,4,216
4,5,44
...,...,...
605,606,1115
606,607,187
607,608,831
608,609,37


In [55]:
mean_rating = ratings.groupby('movieId')[['rating']].mean()
# Lowest rated movies
lowest_rated = mean_rating['rating'].idxmin()
movies.loc[movies['movieId'] == lowest_rated]
# Highest rated movies
highest_rated = mean_rating['rating'].idxmax()
movies.loc[movies['movieId'] == highest_rated]

ratings[ratings['movieId'] == highest_rated]
ratings[ratings['movieId'] == lowest_rated]

movie_stats = ratings.groupby('movieId')[['rating']].agg(['count', 'mean'])
movie_stats.columns = movie_stats.columns.droplevel()

In [56]:
movie_stats

,count,mean
movieId,,
1,215,3.920930
2,110,3.431818
3,52,3.259615
4,7,2.357143
5,49,3.071429
...,...,...
193581,1,4.000000
193583,1,3.500000
193585,1,3.500000


In [73]:
movies.drop_duplicates(subset='movieId', inplace=True)
merge_movies.drop_duplicates(subset='movieId', inplace=True)

In [81]:
imdb_movies.loc[imdb_movies['title'] == 'Waiting to Exhale']

,title,date_x,score,genre,overview,crew,status,orig_lang,budget_x,revenue,country


In [74]:
merge_movies = pd.merge(movies, imdb_movies, on='title', how='left')
merge_movies

,movieId,title,genres,year,date_x,score,genre,overview,crew,status,orig_lang,budget_x,revenue,country
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995.0,12/07/1995,80.0,"Animation, Adventure, Family, Comedy","Led by Woody, Andy's toys live happily in his ...","Tom Hanks, Woody (voice), Tim Allen, Buzz Ligh...",Released,English,30000000.0,3.652710e+08,AU
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995.0,03/21/1996,72.0,"Adventure, Fantasy, Family",When siblings Judy and Peter discover an encha...,"Robin Williams, Alan Parrish, Kirsten Dunst, J...",Released,English,65000000.0,2.627583e+08,AU
2,3,Grumpier Old Men,"[Comedy, Romance]",1995.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Father of the Bride Part II,[Comedy],1995.0,12/08/1995,62.0,"Comedy, Family",Just when George Banks has recovered from his ...,"Steve Martin, George Banks, Diane Keaton, Nina...",Released,English,9540689.0,7.659411e+07,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10281,193581,Black Butler: Book of the Atlantic,"[Action, Animation, Comedy, Fantasy]",2017.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10282,193583,No Game No Life: Zero,"[Animation, Comedy, Fantasy]",2017.0,07/15/2017,79.0,"Animation, Action, Drama","In ancient Disboard, Riku is an angry, young w...","Yoko Hikasa, Couronne / Stephanie, Ai Kayano, ...",Released,Japanese,44401333.2,6.000000e+06,JP
10283,193585,Flint,[Drama],2017.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10284,193587,Bungo Stray Dogs: Dead Apple,"[Action, Animation]",2018.0,03/03/2018,83.0,"Animation, Action, Comedy, Mystery, Crime, Fan...",A large scale catastrophe is occurring across ...,"Kensho Ono, Ryuunosuke Akutagawa, Mamoru Miyan...",Released,Japanese,161000000.0,1.213426e+09,JP


In [80]:
merge_movies.loc[merge_movies['status'].isna()]

,movieId,title,genres,year,date_x,score,genre,overview,crew,status,orig_lang,budget_x,revenue,country
2,3,Grumpier Old Men,"[Comedy, Romance]",1995.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,Tom and Huck,"[Adventure, Children]",1995.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,11,"American President, The","[Comedy, Drama, Romance]",1995.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,14,Nixon,[Drama],1995.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10279,193573,Love Live! The School Idol Movie,[Animation],2015.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10280,193579,Jon Stewart Has Left the Building,[Documentary],2015.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10281,193581,Black Butler: Book of the Atlantic,"[Action, Animation, Comedy, Fantasy]",2017.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10283,193585,Flint,[Drama],2017.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
merge_movies.describe()

,movieId,year,score,budget_x,revenue
count,9730.000000,9729.000000,3367.000000,3.367000e+03,3.367000e+03
mean,42067.419424,1994.613629,66.262845,4.727245e+07,1.666266e+08
std,52040.502726,18.535219,8.157255,4.942196e+07,2.301068e+08
min,1.000000,1902.000000,0.000000,2.600000e+01,0.000000e+00
25%,3244.500000,1988.000000,61.000000,1.190000e+07,2.461553e+07
50%,7283.500000,1999.000000,66.000000,3.000000e+07,7.714703e+07
75%,76048.000000,2008.000000,72.000000,6.624000e+07,2.124170e+08
max,193609.000000,2018.000000,98.000000,3.790000e+08,2.923706e+09


In [77]:
movies.loc[movies.isna().any(axis=1)]

,movieId,title,genres,year
9518,171749,Death Note: Desu nôto,[(no genres listed)],NaN
